<a href="https://colab.research.google.com/github/seulayoon/PyTorch-Tutorial/blob/main/Autograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AUTOGRAD: 자동 미분

autograd 패키지는 Tensor의 모든 연산에 대해 자동 미분을 제공.
실행-기반-정의(define-by-run) 프레임워크로, 코드를 어떻게 작성하여 실행하느냐에 따라 역전파가 정의된다는 뜻이며, 역전파는 학습 과정의 매 단계마다 달라진다.

## Tensor

패키지의 중심에는 torch.Tensor 클래스가 있다. 만약 .requires_grad 속성을 True로  설정하면, 그 tensor에서 이뤄진 모든 연산들을 추적(track)하기 시작한다. 계산이 완료된 후 .backward()를 호출하여 모든 변화도(gradient)를 자동으로 계산할 수 있다. 이 Tensor의 변화도는 .grad 속성에 누적된다.

Tensor가 기록을 추적하는 것을 중단하게 하려면, .detach()를 호출하여 연산 기록으로부터 분리(detach)하여 이후 연산들이 추적되는 것을 방지할 수 있다.

기록을 추적하는 것(과 메모리를 사용하는 것)을 방지하기 위해, 코드 블럭을 withtorch.no_grad(): 로 감쌀 수 있다. 이는 특히 변화도(gradient)는 필요없지만, requries_grad=True가 설정되어 학습 가능한 매개변수를 갖는 모델을 평가(evaluate)할 때 유용하다.

Autograd 구현에서 매우 중요한 클래스는 Function 클래스이다.
Tensor와 Function은 서로 연결되어 있으며, 모든 연산 과정을 부호화(encode)하여 순환하지 않는 그래프(acyclic graph)를 생성한다. 각 tensor는 .grad_fn 속성을 갖고 있는데, 이는 Tensor를 생성한 Function을 참조하고 있다. (단, 사용자가 만든 Tensor는 예외로로, 이 때 grad_fn은 None입니다.)

도함수를 계산하기 위해서는 Tensor의 .backward()를 호출하면 된다. 만약 Tensor가 스칼라인 경우(예. 하나의 요소소 값만 갖는 등)에는 backward에 인자를 정해줄 필요가 없다. 하지만 여러 개의 요소를 갖고 있을 때는 tensor의 모양을 gradient의 인자로 지정할 필요가 있다. 


In [1]:
import torch

In [2]:
x = torch.ones(2, 2, requires_grad=True) # tensor 생성하고 연산을 기록한다.
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [3]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [4]:
print(y.grad_fn)

In [5]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


.requires_grad_( ... ) 는 기존 Tensor의 requires_grad 값을 바꿔치기(in-place)하여 변경한다. 입력값이 지정되지 않으면 기본값은 False이다. 

In [6]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)

a.requires_grad_(True)
print(a.requires_grad)

b = (a * a).sum()
print(b.grad_fn)

False
True


# 변화도(Gradient)

이제 역전파(backprop)를 해보자. out은 하나의 스칼라 값만 가지고 있기 때문에, out.backward()는 out.backward(torch.tensor(1.))과 동일하다.

In [7]:
out.backward()

In [8]:
print(x.grad) # 변화도 d(out)/dx 출력

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


일반적으로, torch.autograd는 벡터-야코비안(Jacobian) 곱을 계산하는 엔진

In [9]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
  y = y * 2

print(y)

tensor([ 734.3218, -575.5486, -785.3188], grad_fn=<MulBackward0>)


이 경우 y는 더 이상 스칼라 값이 아니다. torch.autograd는 전체 야코비안을 직접 계산할 수는 없지만, 벡터-야코비안 곱은 간단히 backward에 해당 벡터를 인자로 제공하여 얻을 수 있다.

In [10]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


In [11]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad(): # autograd가 Tensor들의 연산 기록을 추적하는 것을 멈춤
  print((x ** 2).requires_grad)

True
True
False


.detach()를 호출하여 내용물(content)은 같지만 require_grad가 다른 새로운 Tensor를 가져온다.

In [12]:
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all())

True
False
tensor(True)
